In [14]:
import paramiko
import numpy as np
import itertools
import os
import re

# ===================== SSH CONFIG =====================
SSH_HOST = "10.173.154.77"
SSH_PORT = 22
SSH_USERNAME = "pburton"
SSH_PASSWORD = "pbucf564"

# **REMOTE PATHS
REMOTE_REACTANT_FILE = "/mnt/disk_ssd1/home/pburton/B3LYP/B3LYP2/reactant/RuNMe2OH2O.out"
REMOTE_TS_FILE       = "/mnt/disk_ssd1/home/pburton/B3LYP/B3LYP2/TS/RuNMe2OH2O.out"
REMOTE_PRODUCT_FILE  = "/mnt/disk_ssd1/home/pburton/product/RuNMe2OH2O.out"

# Local names
LOCAL_REACTANT_FILE = "reactant_local.out"
LOCAL_TS_FILE       = "ts_local.out"
LOCAL_PRODUCT_FILE  = "product_local.out"
# Identify atoms based on atomic number
atomic_number_to_symbol = {
    1: "H", 2: "He", 3: "Li", 4: "Be", 5: "B",
    6: "C", 7: "N", 8: "O", 9: "F", 10: "Ne",
    11: "Na", 12: "Mg", 13: "Al", 14: "Si", 15: "P",
    16: "S", 17: "Cl", 18: "Ar", 19: "K", 20: "Ca",
    21: "Sc", 22: "Ti", 23: "V", 24: "Cr", 25: "Mn",
    26: "Fe", 27: "Co", 28: "Ni", 29: "Cu", 30: "Zn",
    31: "Ga", 32: "Ge", 33: "As", 34: "Se", 35: "Br",
    36: "Kr", 37: "Rb", 38: "Sr", 39: "Y", 40: "Zr",
    41: "Nb", 42: "Mo", 43: "Tc", 44: "Ru", 45: "Rh",
    46: "Pd", 47: "Ag", 48: "Cd", 49: "In", 50: "Sn",
    51: "Sb", 52: "Te", 53: "I", 54: "Xe", 55: "Cs",
    56: "Ba", 57: "La", 58: "Ce", 59: "Pr", 60: "Nd",
}

def element_from_atomic_number(num):
    return atomic_number_to_symbol.get(num, str(num))

def download_file_via_ssh(remote_path, local_path):
    try:
        print(f"Connecting via SSH to download '{remote_path}' ...")
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(SSH_HOST, port=SSH_PORT, username=SSH_USERNAME, password=SSH_PASSWORD)
        sftp = ssh.open_sftp()
        sftp.get(remote_path, local_path)
        sftp.close()
        ssh.close()
        print(f"Downloaded {remote_path} -> {local_path}")
    except Exception as e:
        print("Could not download (whomp whomp)", e)

def read_gaussian_coordinates(file_path):
    if not os.path.isfile(file_path):
        return []

    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    block_start = None
    for i, line in enumerate(lines):
        if "Input orientation:" in line:
            block_start = i
            break

    if block_start is None:
        # Search for atoms 'input orientation' in files
        return []

    coord_start = block_start + 5
    coords = []
    for line in lines[coord_start:]:
        stripped = line.strip()
        if stripped.startswith("-----") or not stripped:
            break
        parts = stripped.split()
        if len(parts) >= 6:
            try:
                atomic_num = int(parts[1])
                x = float(parts[3])
                y = float(parts[4])
                z = float(parts[5])
                symbol = element_from_atomic_number(atomic_num)
                coords.append((symbol, x, y, z))
            except ValueError:
                pass
    return coords

def extract_central_metal_charge(file_path):
    if not os.path.isfile(file_path):
        return None

    with open(file_path, "r", encoding='utf-8') as f:
        lines = f.readlines()

    block_start = None
    for i, line in enumerate(lines):
        if ("Population analysis using the SCF Density" in line or
            "Mulliken charges and spin densities" in line):
            block_start = i
            break

    if block_start is None:
        return None

    charges = []
    for line in lines[block_start:]:
        if re.match(r"\s*\d+\s+\w+\s+[-]?\d", line):
            # e.g. "1 Ru  0.915212  1.178459"
            parts = line.split()
            if len(parts) >= 3:
                try:
                    possible_charge = float(parts[2])
                    charges.append(possible_charge)
                except ValueError:
                    pass
        if "Sum of Mulliken charges" or "Mulkien charges" in line:
            break
    metals = {
        "Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Y","Zr","Nb","Mo","Tc","Ru",
        "Rh","Pd","Ag","Cd","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg"
    }
    geo = read_gaussian_coordinates(file_path)
    for i, (sym, x, y, z) in enumerate(geo):
        if sym in metals:
            if i < len(charges):
                return charges[i]
    return None
#==================Ligand Bond Length Threshold Adjustment===============================
def calculate_bond_lengths(atomic_data, threshold=2.3):
    bonds = []
    for (i, at1), (j, at2) in itertools.combinations(enumerate(atomic_data), 2):
        sym1, x1, y1, z1 = at1
        sym2, x2, y2, z2 = at2
        dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
        if dist <= threshold:
            bonds.append((i, j, sym1, sym2, dist))
    return bonds

def extract_connectivity_label_ligands(atomic_data, threshold=2.2):
    bonds = calculate_bond_lengths(atomic_data, threshold=threshold)
    adjacency = {i: [] for i in range(len(atomic_data))}
    for (i, j, sym1, sym2, dist) in bonds:
        adjacency[i].append((j, sym2, dist))
        adjacency[j].append((i, sym1, dist))

#Identify the central metal atom 
    metals = {
        "Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Y","Zr","Nb","Mo","Tc","Ru",
        "Rh","Pd","Ag","Cd","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg"
    }
    metal_index = None
    for idx, (sym, x, y, z) in enumerate(atomic_data):
        if sym in metals:
            metal_index = idx
            break

    ligand_labels = {}
    if metal_index is not None:
        neighbors = adjacency[metal_index]
        if len(neighbors) >= 6:
#Take 6 closest neigboring atoms ================================================================== **this section needs more work**
            neighbors_sorted = sorted(neighbors, key=lambda x: x[2])
            # label them L1..L6
            for i, tup in enumerate(neighbors_sorted[:6], start=1):
                nb_idx, nb_sym, nb_dist = tup
                ligand_labels[nb_idx] = f"L{i}"  # e.g. L1, L2, ...
        else:
            for i, tup in enumerate(neighbors, start=1):
                nb_idx, nb_sym, nb_dist = tup
                ligand_labels[nb_idx] = f"L{i}"

    return adjacency, ligand_labels
#=============================================HOMO-LUMO gap + Free energy==================================================================
def extract_homo_lumo_gap(file_path):
    if not os.path.isfile(file_path):
        return None

    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    alpha_occ = []
    alpha_virt = []
    reading_occ = False
    reading_virt = False

    for line in lines:
        if "Alpha  occ. eigenvalues" in line:
            reading_occ = True
            reading_virt = False
        elif "Alpha virt. eigenvalues" in line:
            reading_occ = False
            reading_virt = True
        elif "Beta  occ. eigenvalues" in line or "Beta virt. eigenvalues" in line:
            reading_occ = False
            reading_virt = False

        if reading_occ or reading_virt:
            parts = line.split("--")
            if len(parts) > 1:
                energies_str = parts[1].strip().split()
                for e_str in energies_str:
                    try:
                        val = float(e_str)
                        if reading_occ:
                            alpha_occ.append(val)
                        else:
                            alpha_virt.append(val)
                    except ValueError:
                        pass

    if alpha_occ and alpha_virt:
        homo = alpha_occ[-1]
        lumo = alpha_virt[0]
        gap_ev = (lumo - homo) * 27.2114
        return gap_ev
    return None

def extract_free_energy(file_path):
    if not os.path.isfile(file_path):
        return None

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if "Sum of electronic and thermal Free Energies=" in line:
                parts = line.split("=")
                if len(parts) == 2:
                    val_str = parts[1].strip()
                    try:
                        return float(val_str)
                    except ValueError:
                        return None
    return None
#==================dipole moment of compound (probably also needs work bc it isn't generating a value for any files I tested)==========
def extract_dipole_moment(file_path):
   
    if not os.path.isfile(file_path):
        return None

    lines = []
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    dipole_block = False
    for line in lines:
        if "Dipole moment" or "debye" or "dipole" or "Dipole" or "(debye)" in line:
            dipole_block = True
            continue

        if dipole_block:
            if "Tot=" in line:
                match = re.search(r"Tot=\s*([\d\.\-]+)", line)
                if match:
                    try:
                        return float(match.group(1))
                    except ValueError:
                        pass
            if not line.strip():
                dipole_block = False
    return None

def parse_and_print_data(label, file_path):
    print(f"\n=== {label.upper()} ===")
    # read geometry
    geo = read_gaussian_coordinates(file_path)
    # metal charge
    m_charge = extract_central_metal_charge(file_path)
    # adjacency + ligands
    adjacency, ligand_labels = extract_connectivity_label_ligands(geo, threshold=2.2)
    # homo-lumo
    gap = extract_homo_lumo_gap(file_path)
    # free energy
    g_val = extract_free_energy(file_path)
    # dipole
    dipole = extract_dipole_moment(file_path)

    print(f"Central Metal Charge: {m_charge}")
    print(f"HOMO-LUMO Gap (eV): {gap}")
    print(f"Free Energy (Hartrees): {g_val}")
    print(f"Dipole Moment: {dipole}")

    metals = {
        "Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Y","Zr","Nb","Mo","Tc","Ru",
        "Rh","Pd","Ag","Cd","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg"
    }
    metal_index = None
    for idx, (sym, x, y, z) in enumerate(geo):
        if sym in metals:
            metal_index = idx
            break

    if metal_index is None:
        print("No metal found (whomp whomp).")
        return

    neighbors = adjacency[metal_index]
    # Sort ligands by distance
    neighbors_sorted = sorted(neighbors, key=lambda x: x[2])
    # Take 6 ligands
    neighbors_six = neighbors_sorted[:6] if len(neighbors_sorted) >= 6 else neighbors_sorted
    print("\nLigands attached to metal (assuming up to 6 neighbors):")
    for i, (nb_idx, nb_sym, nb_dist) in enumerate(neighbors_six, start=1):
        label_i = ligand_labels.get(nb_idx, f"L{i}")
        print(f"  {label_i}: {nb_sym}(index {nb_idx}), distance = {nb_dist:.3f} Å")

def main():
    download_file_via_ssh(REMOTE_REACTANT_FILE, LOCAL_REACTANT_FILE)
    download_file_via_ssh(REMOTE_TS_FILE,       LOCAL_TS_FILE)
    download_file_via_ssh(REMOTE_PRODUCT_FILE,  LOCAL_PRODUCT_FILE)

    # Parsing
    parse_and_print_data("REACTANT", LOCAL_REACTANT_FILE)
    parse_and_print_data("TS",       LOCAL_TS_FILE)
    parse_and_print_data("PRODUCT",  LOCAL_PRODUCT_FILE)

if __name__ == "__main__":
    main()
    
print ("Index is the order in which the ligand appears on the file, for cross-referencing")
print ("Notes: ligand extraction is not yet perfected, still a work in progress")

Connecting via SSH to download '/mnt/disk_ssd1/home/pburton/B3LYP/B3LYP2/reactant/RuNMe2OH2O.out' ...
Downloaded /mnt/disk_ssd1/home/pburton/B3LYP/B3LYP2/reactant/RuNMe2OH2O.out -> reactant_local.out
Connecting via SSH to download '/mnt/disk_ssd1/home/pburton/B3LYP/B3LYP2/TS/RuNMe2OH2O.out' ...
Downloaded /mnt/disk_ssd1/home/pburton/B3LYP/B3LYP2/TS/RuNMe2OH2O.out -> ts_local.out
Connecting via SSH to download '/mnt/disk_ssd1/home/pburton/product/RuNMe2OH2O.out' ...
Downloaded /mnt/disk_ssd1/home/pburton/product/RuNMe2OH2O.out -> product_local.out

=== REACTANT ===
Central Metal Charge: None
HOMO-LUMO Gap (eV): 4.96880164
Free Energy (Hartrees): -783.335867
Dipole Moment: None

Ligands attached to metal (assuming up to 6 neighbors):
  L1: O(index 1), distance = 1.900 Å
  L2: O(index 4), distance = 1.900 Å
  L3: N(index 5), distance = 1.940 Å
  L4: N(index 6), distance = 1.940 Å
  L5: N(index 7), distance = 1.940 Å
  L6: N(index 8), distance = 1.940 Å

=== TS ===
Central Metal Charge: No